In [ ]:
from IPython.display import clear_output
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import re
import dropbox
import sys
import pandas as pd
import numpy as np
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import cv2
import logging
import datetime
import glob
import json
import scipy
import matplotlib as mpl

from subprocess import call
from tifffile import imwrite
from tqdm import tqdm
from pathlib import Path
from amftrack.util.dbx import (
    upload_folder,
    download,
    read_saved_dropbox_state,
    save_dropbox_state,
    load_dbx,
    get_dropbox_folders_prince,
    get_dropbox_video_folders,
    download_video_folders_drop,
    download_analysis_folders_drop,
)
from amftrack.pipeline.launching.run import (
    run_transfer,
)
from amftrack.pipeline.launching.run_super import run_parallel_transfer
from amftrack.pipeline.launching.run_super import run_parallel_flows
from amftrack.pipeline.functions.transport_processing.high_mag_videos.plot_data import (
    plot_summary,
    save_raw_data,
)
from amftrack.pipeline.functions.transport_processing.high_mag_videos.high_mag_analysis import (
    HighmagDataset,
    VideoDataset,
    EdgeDataset,
    index_videos_dropbox_new,
    analysis_run,
)
from amftrack.pipeline.functions.transport_processing.high_mag_videos.kymo_class import (
    KymoVideoAnalysis,
    KymoEdgeAnalysis,
)
from amftrack.util.dbx import upload

%load_ext autoreload
%autoreload 2
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
    save_graphs,
    load_graphs,
    Edge,
    Node,
)
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    reconstruct_image_from_general,
    plot_full,
    plot_edge_color_value,
)
from amftrack.transport.align_video_network import identify_nodes

logging.basicConfig(stream=sys.stdout, level=logging.debug)
%matplotlib widget

In [ ]:
plate_id = "449_20230807"
plate_id_video = "20230818_Plate449"

In [ ]:
analysis_folder = "/projects/0/einf914/analysis_videos/"
# analysis_folder = f"{analysis_folder}{plate_id_video}/"

img_infos = glob.glob(f"{analysis_folder}/**/video_data.json", recursive=True)
vid_anls_frame = pd.DataFrame()
add_infos = []
for address in img_infos:
    add_infos.append(pd.read_json(address, orient="index").T)
vid_anls_frame = pd.concat([vid_anls_frame] + add_infos, ignore_index=True)

vid_anls_frame = vid_anls_frame.sort_values("unique_id").reset_index(drop=True)
vid_anls_frame_select = vid_anls_frame.loc[vid_anls_frame["plate_id"] == plate_id_video]
# vid_anls_frame_select = vid_anls_frame_select[vid_anls_frame_select["video_int"]<=102]
values_id = list(vid_anls_frame_select["unique_id"])

In [ ]:
address

In [ ]:
# img_infos = glob.glob(f"{analysis_folder}/**/video_data.json", recursive=True)
# vid_anls_frame = pd.DataFrame()
# for address in img_infos:
#     add_info = pd.read_json(address, orient="index").T
#     if add_info["plate_id"].iloc[0] == plate_id_video:
#         original_path = add_info["tot_path_drop"].iloc[0]

#         # Replace the specific substring in the target path
#         target_path = original_path.replace(
#             f"/{plate_id_video}", f"/KymoSpeeDExtract/{plate_id_video}"
#         )
#         target = "/" + target_path + "/video_data_network.json"
#         source = address
#         upload(
#             source,
#             target,
#             chunk_size=256 * 1024 * 1024,
#         )
#         print(target)
#     # break

In [ ]:
vid_anls_frame_select["record_time"]

In [ ]:
mpl.rcParams["figure.dpi"] = 100

fig, ax = plt.subplots()
vid_anls_frame_select["timedelta"] = pd.to_timedelta(
    vid_anls_frame_select["record_time"]
)

ax.scatter(vid_anls_frame_select["video_int"], vid_anls_frame_select["timedelta"])

In [ ]:
plates = [plate_id]

In [ ]:
# directory_targ = os.path.join(directory_scratch, "stitch_temp2") + "/"
directory_targ = directory_project
directory_targ = "/projects/0/einf914/transport/"

update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)

In [ ]:
folders = all_folders.loc[all_folders["unique_id"] == plates[0]]
folders = folders.loc[folders["/Analysis/nx_graph_pruned_labeled.p"] == True]

In [ ]:
folders.iloc[i]["folder"]

In [ ]:
folders = folders.sort_values(by="datetime")

exp = Experiment(directory_targ)
i = 111
exp.load(folders.iloc[i : i + 2], suffix="_width")
for t in range(exp.ts):
    exp.load_tile_information(t)

In [ ]:
positions = np.array(vid_anls_frame_select[["ypos", "xpos"]]) * 1000 / 1.725
fig, ax = plt.subplots()
for i, pos in enumerate(positions):
    ax.text(int(pos[1]), -int(pos[0]), str(i))
    ax.scatter(int(pos[1]), -int(pos[0]))

In [ ]:
cid, dicopoint = identify_nodes(exp, 0)

In [ ]:
del dicopoint[""]

In [ ]:
dicopoint

In [ ]:
dicopoint = {
    "34": Node(5224, exp),
    "1": Node(3471, exp),
    "8": Node(1586, exp),
    "9": Node(808, exp),
}

In [ ]:
keys = list(dicopoint.keys())
posnodes = [dicopoint[key].pos(0).tolist() for key in keys]
posvideoslines = [
    vid_anls_frame_select.loc[vid_anls_frame_select["video_int"] == int(key)]
    for key in keys
]
posvideos = [
    [
        posvideosline["ypos"].astype(float).iloc[0] * 1000 / 1.725,
        posvideosline["xpos"].astype(float).iloc[0] * 1000 / 1.725,
    ]
    for posvideosline in posvideoslines
]

In [ ]:
from amftrack.util.image_analysis import find_transformation, find_similarity

transform = find_similarity(posvideos, posnodes)
transform = find_similarity(posvideos, posnodes, ratio=1.128)

similarity_ratio should be 1.128

In [ ]:
posvideos_new = [transform(pos) for pos in posvideos]
posvideos_new, posnodes

In [ ]:
positions = np.array(vid_anls_frame_select[["ypos", "xpos"]]) * 1000 / 1.725
positions_list = positions.tolist()
posvideos_new_list = [transform(pos) for pos in positions_list]

In [ ]:
# analysis_folder = "/projects/0/einf914/analysis_videos/"
# # analysis_folder = f"{analysis_folder}{plate_id_video}/"

# img_infos = glob.glob(f"{analysis_folder}/**/video_data_network.json", recursive=True)
# vid_anls_frame = pd.DataFrame()
# add_infos = []
# for address in img_infos:
#     add_infos.append(pd.read_json(address, orient="index").T)
# vid_anls_frame = pd.concat([vid_anls_frame] + add_infos, ignore_index=True)

# vid_anls_frame = vid_anls_frame.sort_values("unique_id").reset_index(drop=True)
# vid_anls_frame_select = vid_anls_frame.loc[vid_anls_frame["plate_id"] == plate_id_video]
# positions = np.array(vid_anls_frame_select[["ypos_network", "xpos_network"]]) * 1000 / 1.725
# positions_list = positions.tolist()
# posvideos_new_list = [pos for pos in positions_list]

In [ ]:
t = 0
vmax = 9
vmin = 3
region = None
nodes = get_all_nodes(exp, t)
edges = get_all_edges(exp, t)

downsizing = 5

plot_full_video(
    exp,
    t,
    downsizing=downsizing,
    points=posvideos_new_list,
    video_num=list(vid_anls_frame_select["video_int"]),
    edges=edges,
    dilation=5,
    figsize=(12, 6),
)

In [ ]:
analysis_folder

In [ ]:
img_infos = glob.glob(
    f"{analysis_folder}/CocoTransport/{plate_id_video}/**/video_data.json",
    recursive=True,
)
for address in img_infos:
    add_info = pd.read_json(address, orient="index").T
    if add_info["plate_id"].iloc[0] == plate_id_video:
        if add_info["unique_id"].iloc[0] in values_id:
            positions = np.array(add_info[["ypos", "xpos"]]) * 1000 / 1.725
            positions_list = positions.tolist()
            posvideos_new_list = [transform(pos) for pos in positions_list]
            add_info["xpos_network"] = posvideos_new_list[0][0]
            add_info["ypos_network"] = posvideos_new_list[0][1]
            add_info.to_json(address)
        # break

In [ ]:
img_infos = glob.glob(
    f"{analysis_folder}/CocoTransport/{plate_id_video}/**/video_data.json",
    recursive=True,
)
vid_anls_frame = pd.DataFrame()
for address in img_infos:
    add_info = pd.read_json(address, orient="index").T
    if add_info["plate_id"].iloc[0] == plate_id_video:
        if add_info["unique_id"].iloc[0] in values_id:

            original_path = add_info["tot_path_drop"].iloc[0]

            # Replace the specific substring in the target path
            target_path = original_path.replace(
                f"/{plate_id_video}", f"/KymoSpeeDExtract/{plate_id_video}"
            )
            target = "/" + target_path + "/video_data_network.json"
            source = address
            upload(
                source,
                target,
                chunk_size=256 * 1024 * 1024,
            )
            print(target)
    # break

In [ ]:
t = 0
im, skel_im = make_full_image(
    exp, t, downsizing=5, dilation=5, edges=get_all_edges(exp, t)
)

In [ ]:
# im_rgba = np.stack([im, im, im, np.ones(im.shape, dtype=im.dtype) * 255], axis=-1)
# white_mask = np.all(skel_im[:, :, :3] == 255, axis=-1)
# skel_im[white_mask, 3] = 0  # setting alpha to 0 where it's white
# # Convert numpy arrays to PIL images
# im_pil = Image.fromarray(im_rgba.astype(np.uint8))
# skel_im_pil = Image.fromarray(skel_im.astype(np.uint8))


# Overlay the images
# combined = Image.alpha_composite(im_pil, skel_im_pil)
combined.save(os.path.join(analysis_folder, "network_overlay.png"))
im_pil.save(os.path.join(analysis_folder, "stitched.png"))

fig, ax = plt.subplots(figsize=(8, 6))
ax.imshow(combined)

In [ ]:
if im_pil.size[1] < skel_im_pil.size[1]:
    # Create a new image with one more pixel in height
    new_im = Image.new("RGBA", (im_pil.width, im_pil.height + 1), (255, 255, 255, 0))
    # Paste the original image into the new image
    new_im.paste(im_pil, (0, 0))
else:
    # Create a new image with one more pixel in height for skel_im_pil if needed
    new_im = Image.new(
        "RGBA", (skel_im_pil.width, skel_im_pil.height + 1), (255, 255, 255, 0)
    )
    # Paste the original skel_im_pil into the new image
    new_im.paste(skel_im_pil, (0, 0))
    # Since we modified skel_im_pil, we'll use it in the composition
    skel_im_pil = new_im

# Now you can use Image.alpha_composite if im_pil was modified
if im_pil.size[1] < skel_im_pil.size[1]:
    combined = Image.alpha_composite(new_im, skel_im_pil)
else:
    combined = Image.alpha_composite(im_pil, skel_im_pil)

In [ ]:
source = os.path.join(analysis_folder, "network_overlay.png")
original_path = vid_anls_frame_select["tot_path_drop"].iloc[0]
new_path = os.path.dirname(os.path.dirname(original_path))
new_path = "/" + new_path
target = new_path + "/network_overlay.png"
target = target.replace(f"/{plate_id_video}", f"/Analysis/{plate_id_video}")

upload(
    source,
    target,
    chunk_size=256 * 1024 * 1024,
)
source = os.path.join(analysis_folder, "stitched.png")

target = new_path + "/stitched.png"
target = target.replace(f"/{plate_id_video}", f"/Analysis/{plate_id_video}")

upload(
    source,
    target,
    chunk_size=256 * 1024 * 1024,
)

In [ ]:
plate_id_video

In [ ]:
def plot_full_video(
    exp: Experiment,
    t: int,
    region=None,
    edges: List[Edge] = [],
    points: List[coord_int] = [],
    video_num: List[int] = [],
    segments: List[List[coord_int]] = [],
    nodes: List[Node] = [],
    downsizing=5,
    dilation=1,
    save_path="",
    prettify=False,
    with_point_label=False,
    figsize=(12, 8),
    dpi=None,
    node_size=5,
) -> None:
    """
    This is the general purpose function to plot the full image or a region `region` of the image at
    any given timestep t. The region being specified in the GENERAL coordinates.
    The image can be downsized by a chosen factor `downsized` with additionnal features such as: edges, nodes, points, segments.
    The coordinates for all the objects are provided in the GENERAL referential.

    :param region: choosen region in the full image, such as [[100, 100], [2000,2000]], if None the full image is shown
    :param edges: list of edges to plot, it is the pixel list that is plotted, not a straight line
    :param nodes: list of nodes to plot (only nodes in the `region` will be shown)
    :param points: points such as [123, 234] to plot with a red cross on the image
    :param segments: plot lines between two points that are provided
    :param downsizing: factor by which we reduce the image resolution (5 -> image 25 times lighter)
    :param dilation: only for edges: thickness of the edges (dilation applied to the pixel list)
    :param save_path: full path to the location where the plot will be saved
    :param prettify: if True, the image will be enhanced by smoothing the intersections between images
    :param with_point_label: if True, the index of the point is ploted on top of it

    NB: the full region of a full image is typically [[0, 0], [26000, 52000]]
    NB: the interesting region of a full image is typically [[12000, 15000], [26000, 35000]]
    NB: the colors are chosen randomly for edges
    NB: giving a smaller region greatly increase computation time
    """

    # TODO(FK): fetch image size from experiment object here, and use it in reconstruct image
    # TODO(FK): colors for edges are not consistent
    # NB: possible other parameters that could be added: alpha between layers, colors for object, figure_size
    DIM_X, DIM_Y = get_dimX_dimY(exp)

    if region == None:
        # Full image
        image_coodinates = exp.image_coordinates[t]
        region = get_bounding_box(image_coodinates)
        region[1][0] += DIM_X  # TODO(FK): Shouldn't be hardcoded
        region[1][1] += DIM_Y

    # 1/ Image layer
    im, f = reconstruct_image_from_general(
        exp,
        t,
        downsizing=downsizing,
        region=region,
        prettify=prettify,
        white_background=False,  # TODO(FK): add image dimention here dimx = ..
    )
    f_int = lambda c: f(c).astype(int)
    new_region = [
        f_int(region[0]),
        f_int(region[1]),
    ]  # should be [[0, 0], [d_x/downsized, d_y/downsized]]

    # 2/ Edges layer
    skel_im, _ = reconstruct_skeletton(
        [edge.pixel_list(t) for edge in edges],
        region=region,
        color_seeds=[(edge.begin.label + edge.end.label) % 255 for edge in edges],
        downsizing=downsizing,
        dilation=dilation,
    )

    # 3/ Fusing layers
    fig = plt.figure(
        figsize=figsize
    )  # width: 30 cm height: 20 cm # TODO(FK): change dpi
    ax = fig.add_subplot(111)
    ax.imshow(im, cmap="gray", interpolation="none")
    ax.imshow(skel_im, alpha=0.5, interpolation="none")

    # 3/ Plotting the Nodes
    size = node_size
    for node in nodes:
        c = f(list(node.pos(t)))
        color = make_random_color(node.label)[:3]
        reciprocal_color = 255 - color
        color = tuple(color / 255)
        reciprocal_color = tuple(reciprocal_color / 255)
        bbox_props = dict(boxstyle="circle", fc=color, edgecolor="none")
        if is_in_bounding_box(c, new_region):
            node_text = ax.text(
                c[1],
                c[0],
                str(node.label),
                ha="center",
                va="center",
                bbox=bbox_props,
                font=fpath,
                fontdict={"color": reciprocal_color},
                size=size,
                # alpha = 0.5
            )
    # 4/ Plotting coordinates
    points = [f(c) for c in points]
    for i, c in enumerate(points):
        if is_in_bounding_box(c, new_region):
            color = make_random_color(video_num[i])[:3]
            color = tuple(color / 255)
            plt.text(c[1], c[0], video_num[i], color="black", fontsize=20, alpha=1)
            plt.plot(c[1], c[0], marker="x", color="black", markersize=10, alpha=0.5)

            if with_point_label:
                plt.text(c[1], c[0], f"{i}")

    # 5/ Plotting segments
    segments = [[f(segment[0]), f(segment[1])] for segment in segments]
    for s in segments:
        plt.plot(
            [s[0][1], s[1][1]],  # x1, x2
            [s[0][0], s[1][0]],  # y1, y2
            color="white",
            linewidth=2,
        )

    if save_path:
        plt.savefig(save_path, dpi=dpi)
    else:
        plt.show()
    return ax

In [ ]:
from amftrack.pipeline.functions.image_processing.experiment_util import *
from PIL import Image


def make_full_image(
    exp: Experiment,
    t: int,
    region=None,
    edges: List[Edge] = [],
    points: List[coord_int] = [],
    video_num: List[int] = [],
    segments: List[List[coord_int]] = [],
    nodes: List[Node] = [],
    downsizing=5,
    dilation=1,
    save_path="",
    prettify=False,
    with_point_label=False,
    figsize=(12, 8),
    dpi=None,
    node_size=5,
) -> None:
    """
    This is the general purpose function to plot the full image or a region `region` of the image at
    any given timestep t. The region being specified in the GENERAL coordinates.
    The image can be downsized by a chosen factor `downsized` with additionnal features such as: edges, nodes, points, segments.
    The coordinates for all the objects are provided in the GENERAL referential.

    :param region: choosen region in the full image, such as [[100, 100], [2000,2000]], if None the full image is shown
    :param edges: list of edges to plot, it is the pixel list that is plotted, not a straight line
    :param nodes: list of nodes to plot (only nodes in the `region` will be shown)
    :param points: points such as [123, 234] to plot with a red cross on the image
    :param segments: plot lines between two points that are provided
    :param downsizing: factor by which we reduce the image resolution (5 -> image 25 times lighter)
    :param dilation: only for edges: thickness of the edges (dilation applied to the pixel list)
    :param save_path: full path to the location where the plot will be saved
    :param prettify: if True, the image will be enhanced by smoothing the intersections between images
    :param with_point_label: if True, the index of the point is ploted on top of it

    NB: the full region of a full image is typically [[0, 0], [26000, 52000]]
    NB: the interesting region of a full image is typically [[12000, 15000], [26000, 35000]]
    NB: the colors are chosen randomly for edges
    NB: giving a smaller region greatly increase computation time
    """

    # TODO(FK): fetch image size from experiment object here, and use it in reconstruct image
    # TODO(FK): colors for edges are not consistent
    # NB: possible other parameters that could be added: alpha between layers, colors for object, figure_size
    DIM_X, DIM_Y = get_dimX_dimY(exp)

    if region == None:
        # Full image
        image_coodinates = exp.image_coordinates[t]
        region = get_bounding_box(image_coodinates)
        region[1][0] += DIM_X  # TODO(FK): Shouldn't be hardcoded
        region[1][1] += DIM_Y

    # 1/ Image layer
    im, f = reconstruct_image_from_general(
        exp,
        t,
        downsizing=downsizing,
        region=region,
        prettify=prettify,
        white_background=False,  # TODO(FK): add image dimention here dimx = ..
    )
    f_int = lambda c: f(c).astype(int)
    new_region = [
        f_int(region[0]),
        f_int(region[1]),
    ]  # should be [[0, 0], [d_x/downsized, d_y/downsized]]

    # 2/ Edges layer
    skel_im, _ = reconstruct_skeletton(
        [edge.pixel_list(t) for edge in edges],
        region=region,
        color_seeds=[(edge.begin.label + edge.end.label) % 255 for edge in edges],
        downsizing=downsizing,
        dilation=dilation,
    )
    return (im, skel_im)

In [ ]:
np.max(skel_im)